In [6]:
import json
import pandas as pd

train_list = []
file_name = "train_2024_S2.txt"
with open(file_name, "r", encoding="utf-8") as f:
    for line in f:
        if line[0] in ["G", "D", "C", "S", "Z", "Y", "T", "K", "N"]:
            train_list.append({"train_id": "", "train_time": []})
            train_list[-1]["train_id"] = line.strip()
        elif line == "" or line == "\n":
            continue
        else:
            train_list[-1]["train_time"].append(line.strip().split("\t"))

with open("train_time.json", "w", encoding="utf-8") as f:
    json.dump(train_list, f, ensure_ascii=False, indent=4)

In [7]:
train_list

[{'train_id': 'G14',
  'train_time': [['上海', '11:53', '11:53'],
   ['常州北', '12:37', '12:39'],
   ['南京南', '13:08', '13:10'],
   ['济南西', '15:08', '15:10'],
   ['北京南', '16:38', '16:38']]},
 {'train_id': 'G15',
  'train_time': [['北京南', '14:00', '14:00'],
   ['济南西', '15:23', '15:25'],
   ['南京南', '17:22', '17:24'],
   ['苏州北', '18:08', '18:10'],
   ['上海虹桥', '18:33', '18:33']]},
 {'train_id': 'G4',
  'train_time': [['苏州北', '07:24', '07:24'],
   ['无锡东', '07:34', '07:36'],
   ['南京南', '08:15', '08:17'],
   ['济南西', '10:15', '10:17'],
   ['北京南', '11:40', '11:40']]},
 {'train_id': 'G9',
  'train_time': [['北京南', '11:00', '11:00'],
   ['济南西', '12:22', '12:24'],
   ['徐州东', '13:21', '13:23'],
   ['南京南', '14:32', '14:34'],
   ['上海虹桥', '15:37', '15:37']]},
 {'train_id': 'G13',
  'train_time': [['北京南', '13:00', '13:00'],
   ['天津南', '13:31', '13:33'],
   ['济南西', '14:30', '14:32'],
   ['南京南', '16:30', '16:32'],
   ['上海', '17:35', '17:35']]},
 {'train_id': 'G7',
  'train_time': [['北京南', '10:00', '10:00'],
   

### 为火车站编号

In [8]:
s = set()
for train in train_list:
    for stop in train["train_time"]:
        s.add(stop[0])
total = len(s)

df = pd.DataFrame(s).sort_values(by=0).reset_index(drop=True)
df.columns = ["station"]
df["station_id"] = range(0, total)
d={}
for i in range(total):
    d[df["station"][i]] = str(df["station_id"][i])
with open("station_id.json", "w", encoding="utf-8") as f:
    json.dump(d, f, ensure_ascii=False, indent=4)

In [9]:
station2id = {}
for id in range(0, total):
    station2id[df.iloc[id, 0]] = id

for train in train_list:
    for stop in train["train_time"]:
        stop.append(station2id[stop[0]])

with open("train_time.json", "w", encoding="utf-8") as f:
    json.dump(train_list, f, ensure_ascii=False, indent=4)

In [10]:
d

{'4317': '0',
 '4318': '1',
 '4355': '2',
 '4356': '3',
 '4357': '4',
 '4358': '5',
 '4367': '6',
 '4368': '7',
 '7523': '8',
 '7524': '9',
 '7557': '10',
 '7558': '11',
 '一步滩': '12',
 '一面坡': '13',
 '一面坡北': '14',
 '七台河': '15',
 '七台河西': '16',
 '七甸': '17',
 '七苏木': '18',
 '七营': '19',
 '七龙星': '20',
 '万乐': '21',
 '万发屯': '22',
 '万宁': '23',
 '万安县': '24',
 '万州': '25',
 '万州北': '26',
 '万年': '27',
 '万源': '28',
 '三义井': '29',
 '三亚': '30',
 '三元坝': '31',
 '三元西': '32',
 '三关口': '33',
 '三十家': '34',
 '三原': '35',
 '三合庄': '36',
 '三家寨': '37',
 '三家店': '38',
 '三明': '39',
 '三明北': '40',
 '三水北': '41',
 '三水南': '42',
 '三汇镇': '43',
 '三江': '44',
 '三江南': '45',
 '三江口': '46',
 '三河县': '47',
 '三源浦': '48',
 '三穗': '49',
 '三花石': '50',
 '三营': '51',
 '三道营': '52',
 '三都县': '53',
 '三门县': '54',
 '三门峡': '55',
 '三门峡南': '56',
 '三门峡西': '57',
 '三间房': '58',
 '三阳': '59',
 '三阳川': '60',
 '上万': '61',
 '上仓': '62',
 '上普雄': '63',
 '上杭': '64',
 '上板城': '65',
 '上板城南': '66',
 '上海': '67',
 '上海南': '68',
 '上海虹桥': '69',
 '上海西': '70',
 '上腰墩': '71',
 '

### 建图

In [11]:
import datetime

nodes = []

for d in train_list:
    data = d['train_time']
    for i in range(len(data)):
        nodes.append((data[i][0], 0, data[i][1]))
        nodes.append((data[i][0], 1, data[i][2]))

# 去重 + 排序（首先按第一维排序，然后按第二维排序，最后按第三维排序）
nodes = list(set(nodes))
nodes.sort(key=lambda x: (x[0], x[1], x[2]))
print(f"Total nodes: {len(nodes)}")

st_dict, len_dict, sta2id = {}, {}, {}
for i in range(len(nodes)):
    if (nodes[i][0],nodes[i][1]) not in st_dict:
        st_dict[(nodes[i][0],nodes[i][1])] = i
        len_dict[(nodes[i][0],nodes[i][1])] = 1
    else:
        len_dict[(nodes[i][0],nodes[i][1])] += 1

assert len(st_dict) == len(len_dict)

for i in range(len(nodes)):
    sta2id[nodes[i]] = i

file = open("graph.txt", "w", encoding="utf-8")
file.write(f"{len(nodes)} {len(st_dict)}\n")
for key, value in st_dict.items():
    file.write(f"{key[0]} {key[1]} {value} {len_dict[key]}\n")

for node in nodes:
    minutes = datetime.datetime.strptime(node[2], "%H:%M").hour * 60 + datetime.datetime.strptime(node[2], "%H:%M").minute
    file.write(f"{minutes}\n")

for train in train_list:
    for f in range(0, len(train["train_time"]) - 1):
        for t in range(f + 1, len(train["train_time"])):
            starttime = datetime.datetime.strptime(train["train_time"][f][2], "%H:%M")
            midtime = datetime.datetime.strptime(
                train["train_time"][(f + t) // 2][2], "%H:%M"
            )
            endtime = datetime.datetime.strptime(train["train_time"][t][1], "%H:%M")
            during = int((endtime - midtime).seconds / 60) + int(
                (midtime - starttime).seconds / 60
            )
            Y = False
            if endtime < starttime or during >= 24 * 60:
                Y = True
            f_station = sta2id[(train["train_time"][f][0], 1, train["train_time"][f][2])]
            t_station = sta2id[(train["train_time"][t][0], 0, train["train_time"][t][1])]

            if Y:
                file.write(
                    str(f_station)
                    + " "
                    + str(t_station)
                    + " "
                    + str(during)
                    + " "
                    + train["train_id"]
                    + "："
                    + train["train_time"][f][0]
                    + "至"
                    + train["train_time"][t][0]
                    + "从"
                    + train["train_time"][f][2]
                    + "到"
                    + train["train_time"][t][1]
                    + " Y\n"
                )
            else:
                file.write(
                    str(f_station)
                    + " "
                    + str(t_station)
                    + " "
                    + str(during)
                    + " "
                    + train["train_id"]
                    + "："
                    + train["train_time"][f][0]
                    + "至"
                    + train["train_time"][t][0]
                    + "从"
                    + train["train_time"][f][2]
                    + "到"
                    + train["train_time"][t][1]
                    + " D\n"
                )
            

for key in station2id.keys():
    for i in range(st_dict[(key, 0)], st_dict[(key, 0)] + len_dict[(key, 0)]):
        for j in range(st_dict[(key, 1)], st_dict[(key, 1)] + len_dict[(key, 1)]):
            # 将nodes[i]到nodes[j]的时间转化为datetime
            starttime = datetime.datetime.strptime(nodes[i][2], "%H:%M")
            endtime = datetime.datetime.strptime(nodes[j][2], "%H:%M")
            # 计算时间差
            during = int((endtime - starttime).seconds / 60)
            if during <= 360 and during >= 15:
                if endtime < starttime:
                    file.write(f"{i} {j} {during} W Y\n")
                else:
                    file.write(f"{i} {j} {during} W D\n")

with open("city_list.txt", "r") as f:
    for line in f:
        city = line.split(" ")
        if city[0] in station2id and city[1] in station2id:
            for i in range(st_dict[(city[0], 0)], st_dict[(city[0], 0)] + len_dict[(city[0], 0)]):
                for j in range(st_dict[(city[1], 1)], st_dict[(city[1], 1)] + len_dict[(city[1], 1)]):
                    starttime = datetime.datetime.strptime(nodes[i][2], "%H:%M")
                    endtime = datetime.datetime.strptime(nodes[j][2], "%H:%M")
                    during = int((endtime - starttime).seconds / 60)
                    if during < 360 and during > int(city[2]):
                        if endtime < starttime:
                            file.write(f"{i} {j} {during} T Y\n")
                        else:
                            file.write(f"{i} {j} {during} T D\n")
            for i in range(st_dict[(city[1], 0)], st_dict[(city[1], 0)] + len_dict[(city[1], 0)]):
                for j in range(st_dict[(city[0], 1)], st_dict[(city[0], 1)] + len_dict[(city[0], 1)]):
                    starttime = datetime.datetime.strptime(nodes[i][2], "%H:%M")
                    endtime = datetime.datetime.strptime(nodes[j][2], "%H:%M")
                    during = int((endtime - starttime).seconds / 60)
                    if during < 360 and during > int(city[2]):
                        if endtime < starttime:
                            file.write(f"{i} {j} {during} T Y\n")
                        else:
                            file.write(f"{i} {j} {during} T D\n")
                            

Total nodes: 248852


In [12]:
import datetime, math

INTERVAL = 15
SPLIT = 24 * 60 // INTERVAL
STATION_END = total * SPLIT * 2
LONGEST_TIME = 6 * 60 // INTERVAL


def station_in(station_id):
    return station_id * SPLIT * 2


def station_out(station_id):
    return station_id * SPLIT * 2 + SPLIT


def transfer(station_id, time, io):
    time = datetime.datetime.strptime(time, "%H:%M")
    if io == 0:
        time = math.ceil((time.hour * 60 + time.minute) / 15) % SPLIT
    else:
        time = math.floor((time.hour * 60 + time.minute) / 15) % SPLIT
    return int(station_id * SPLIT * 2 + time + io * SPLIT)


file = open("in.txt", "w")
file.write(str(len(d.keys())) + "\n")
for i, v in d.items():
    file.write(str(i) + " " + str(v) + "\n")
for train in train_list:
    for f in range(0, len(train["train_time"]) - 1):
        for t in range(f + 1, len(train["train_time"])):
            starttime = datetime.datetime.strptime(train["train_time"][f][2], "%H:%M")
            midtime = datetime.datetime.strptime(
                train["train_time"][(f + t) // 2][2], "%H:%M"
            )
            endtime = datetime.datetime.strptime(train["train_time"][t][1], "%H:%M")
            during = int((endtime - midtime).seconds / 60) + int(
                (midtime - starttime).seconds / 60
            )
            Y = False
            if endtime < starttime or during >= 24 * 60:
                Y = True
            f_station = transfer(
                train["train_time"][f][3], train["train_time"][f][2], 1
            )
            t_station = transfer(
                train["train_time"][t][3], train["train_time"][t][1], 0
            )

            if Y:
                file.write(
                    str(f_station)
                    + " "
                    + str(t_station)
                    + " "
                    + str(during)
                    + " "
                    + train["train_id"]
                    + "："
                    + train["train_time"][f][0]
                    + "至"
                    + train["train_time"][t][0]
                    + "从"
                    + train["train_time"][f][2]
                    + "到"
                    + train["train_time"][t][1]
                    + " Y\n"
                )
            else:
                file.write(
                    str(f_station)
                    + " "
                    + str(t_station)
                    + " "
                    + str(during)
                    + " "
                    + train["train_id"]
                    + "："
                    + train["train_time"][f][0]
                    + "至"
                    + train["train_time"][t][0]
                    + "从"
                    + train["train_time"][f][2]
                    + "到"
                    + train["train_time"][t][1]
                    + " D\n"
                )

for i in range(total):
    for j in range(station_in(i), station_out(i)):
        for k in range(j + 1, j + LONGEST_TIME):
            if k + SPLIT < (i + 1) * SPLIT * 2:
                file.write(
                    str(j)
                    + " "
                    + str(k + SPLIT)
                    + " "
                    + str((k - j) * INTERVAL)
                    + " W D\n"
                )
            else:
                file.write(
                    str(j) + " " + str(k) + " " + str((k - j) * INTERVAL) + " W Y\n"
                )

with open("city_list.txt", "r") as f:
    for line in f:
        city = line.split(" ")
        if city[0] in d and city[1] in d:
            fr, to = station_in(int(d[city[0]])), station_in(int(d[city[1]]))
            for i in range(fr, fr + SPLIT):
                for j in range(
                    to + math.ceil(int(city[2]) / INTERVAL), to + LONGEST_TIME
                ):
                    if j + SPLIT < (int(d[city[1]]) + 1) * SPLIT * 2:
                        file.write(
                            str(i)
                            + " "
                            + str(j + SPLIT)
                            + " "
                            + str((j - to) * INTERVAL)
                            + " T D\n"
                        )
                    else:
                        file.write(
                            str(i)
                            + " "
                            + str(j)
                            + " "
                            + str((j - to) * INTERVAL)
                            + " T Y\n"
                        )
            fr, to = station_in(int(d[city[1]])), station_in(int(d[city[0]]))
            for i in range(fr, fr + SPLIT):
                for j in range(
                    to + i - fr + math.ceil(int(city[2]) / INTERVAL),
                    to + i - fr + +LONGEST_TIME,
                ):
                    if j + SPLIT < (int(d[city[0]]) + 1) * SPLIT * 2:
                        file.write(
                            str(i)
                            + " "
                            + str(j + SPLIT)
                            + " "
                            + str((j - (to + i - fr)) * INTERVAL)
                            + " T D\n"
                        )
                    else:
                        file.write(
                            str(i)
                            + " "
                            + str(j)
                            + " "
                            + str((j - (to + i - fr)) * INTERVAL)
                            + " T Y\n"
                        )